In [1]:
import os
from crewai import Agent, Crew, Task, LLM, Process
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from dotenv import load_dotenv

/opt/anaconda3/envs/agents/lib/python3.10/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/opt/anaconda3/envs/agents/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/opt/anaconda3/envs/agents/lib/python3.10/site-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. 

In [2]:
load_dotenv()
CEREBRAS_API_KEY = os.getenv("CEREBRAS_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

In [3]:
if not CEREBRAS_API_KEY:
    raise ValueError("Missing Cerebras API Key! Set CEREBRAS_API_KEY in environment variables.")

if not SERPER_API_KEY:
    raise ValueError("Missing Serper API Key! Set SERPER_API_KEY in environment variables.")

In [4]:
cerebras_llm = LLM(
    model="cerebras/llama-3.3-70b",
    temperature=0.7,
    max_tokens=8192,
    api_key=CEREBRAS_API_KEY,
    base_url="https://api.cerebras.ai/v1",
)

### Tools

In [5]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

### Agents

In [6]:
search = Agent(
    role="Product Price Finder",
    goal="Accurately find and list product prices from reputable e-commerce websites.",
    backstory="An expert e-commerce analyst who specializes in finding the best prices from multiple platforms.",
    tools=[search_tool, scrape_tool],
    verbose=True,
    llm=cerebras_llm
)

data_cleaner = Agent(
    role="Data Cleaning & Normalization Specialist",
    goal="Ensure accurate price data by filtering out incorrect entries and converting currencies.",
    backstory="A meticulous data specialist with expertise in filtering noise and ensuring accurate price data.",
    tools=[], 
    verbose=True,
    llm=cerebras_llm
)

comparison = Agent(
    role="Price Comparison Specialist",
    goal="Compare product prices from different e-commerce platforms and return the best deal.",
    backstory="An analytical specialist who finds the best product deals based on price, quality, and reviews.",
    tools=[],  
    verbose=True,
    llm=cerebras_llm
)

reporting_agent = Agent(
    role="Final Report Generator",
    goal="Summarize the best product deal and generate a structured comparison report.",
    backstory="A data-driven analyst who creates structured reports highlighting the best product deals.",
    tools=[], 
    verbose=True,
    llm=cerebras_llm
)

### Tasks 

In [7]:
search_task = Task(
    description="Find all prices for the product from trusted e-commerce websites.",
    expected_output="A structured list of product prices with store links.",
    human_input=True,
    agent=search,
)

cleaning_task = Task(
    description="Filter out incorrect product prices, remove duplicates, and convert to the correct currency.",
    expected_output="A cleaned dataset with accurate prices.",
    human_input=False,
    agent=data_cleaner, 
)

comparison_task = Task(
    description="Compare prices across different platforms and highlight the best deal.",
    expected_output="A ranked list of the best prices with recommendations.",
    human_input=False,
    agent=comparison, 
)

reporting_task = Task(
    description="Generate a user-friendly price comparison report.",
    expected_output="A structured report with the best-reviewed options and store links.",
    human_input=False,
    agent=reporting_agent,
)

In [8]:
product_price_crew = Crew(
    agents=[search, data_cleaner, comparison, reporting_agent],
    tasks=[search_task, cleaning_task, comparison_task, reporting_task], 
    verbose=True,
    process=Process.sequential,
)

In [ ]:
product_name = "Sony WH-1000XM5"
country = "United States"
# format = {'product': product_name, 'country': country}
format = {'product': product_name}

In [15]:
# Execute Crew
event_analysis = product_price_crew.kickoff(inputs=format)
# Print the final report
print(event_analysis)

# Agent: Product Price Finder
## Task: Find all prices for the product from trusted e-commerce websites.


# Agent: Product Price Finder
## Thought: Thought: To find all prices for the product from trusted e-commerce websites, I need to start by searching the internet for the product. This will give me a list of relevant websites that sell the product, and then I can visit each website to find the price.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"product prices\"}"
## Tool Output: 
{'searchParameters': {'q': 'product prices', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Google Shopping - Shop Online, Compare Prices & Where to Buy', 'link': 'https://shopping.google.com/', 'snippet': "Browse Google Shopping to find the products you're looking for, track & compare prices, and decide where to buy online or in store.", 'position': 1}, {'title': 'camelcamelcamel.com: Amazon US price tracker, price history, price ...', 'lin

BadRequestError: litellm.BadRequestError: CerebrasException - Error code: 400 - {'message': 'Please reduce the length of the messages or completion. Current length is 10772 while limit is 8192', 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}